# 4 より高度な分析1：日本人の米離れは本当か？

<a class="anchor" id="contents"></a>
### 目次
- [4.1 オープンデータを使ってみよう](#section04-1)
 - [コード4.1 家計調査年報の読み込み](#code04-1)
- [4.2 米の支出金額は減少しているのか？](#section04-2)
 - [a. 他項目と比較しないケース：米の支出金額の時系列分析](#section04-2a)
   - [コード4.2 米の支出金額の要約統計量の出力](#code04-2)
   - [コード4.3 米の支出金額の分布](#code04-3)
   - [コード4.4 米の支出金額の経年変化のグラフ描画](#code04-4)
   - [コード4.5 米の支出金額の変化率の計算](#code04-5)
 - [b. 他項目と比較を行うケース：米と他の項目との消費支出の傾向比較](#section04-2b)
   - [コード4.6 食料と支出金額の集計処理](#code04-6)
   - [コード4.7 穀類の支出金額の集計処理](#code04-7)
   - [米と他項目との比較](#section04-2b_comp)
     - [コード4.8 時系列データの基礎的な分析の関数化](#code04-8)
     - [コード4.9 関数 analyze を使った基礎的な分析](#code04-9)
     - [コード4.10 変化率を比較するグラフの描画](#code04-10)
- [4.3 来年はどうなるの？](#section04-3)
 - [a. 移動平均法を用いた傾向の可視化](#section04-3a)
   - [コード4.11 支出金額の移動平均の計算](#code04-11)
   - [コード4.12 支出金額の変化率の移動平均の計算](#code04-12)
 - [b. 指数平滑法を用いた予測モデルの構築](#section04-3b)
   - [コード4.13 指数平滑法による予測値を計算する関数](#code04-13)
   - [コード4.14 係数αごとの予測誤差の合計値の計算](#code04-14)
   - [コード4.15 次の年の支出金額の予測](#code04-15)

<a class="anchor" id="section04-1"></a>
### 4.1 オープンデータを使ってみよう

<a class="anchor" id="code04-1"></a>
#### コード4.1 家計調査年報データの読み込み

In [ ]:
import pandas as pd  # 表形式のデータを扱うために pandas を使用する
kakei = pd.read_csv("kakei.csv", encoding="Shift_JIS")  # 家計調査年報データを読み込む
kakei.head(12)  # 読み込んだデータの先頭から12行を表示

[目次へ戻る](#contents)

---

<a class="anchor" id="section04-2"></a>
## 4.2 米の支出金額は減少しているのか？

<a class="anchor" id="section04-2a"></a>
**a. 他項目と比較しないケース：米の支出金額の時系列分析**

<a class="anchor" id="code04-2"></a>
#### コード4.2 米の支出金額の要約統計量の出力

In [ ]:
import pandas as pd
kakei = pd.read_csv("kakei.csv", encoding="Shift_JIS")

# 米の支出金額の行を選択する
rice = kakei[kakei["収支分類区分3"] == "米"]
#print(rice)  # 選択したデータの確認時はコメントアウトを外すこと

# 要約統計量を出力する
rice.loc[:, "支出金額"].describe()

[目次へ戻る](#contents)

<a class="anchor" id="code04-3"></a>
#### コード4.3 米の支出金額の分布

In [ ]:
import matplotlib.pyplot as plt # グラフを作成するために matplotlib を使用する
import japanize_matplotlib

# データの分布として支出金額のヒストグラムを描画する
plt.hist(rice["支出金額"], bins=5, rwidth=0.9) # bins は分布のグループ数、rwidth は棒の横幅
plt.title("米の支出金額のヒストグラム")
plt.xlabel("支出金額（円）")
plt.ylabel("度数")
plt.show()

[目次へ戻る](#contents)

<a class="anchor" id="code04-4"></a>
#### コード4.4 米の支出金額の経年変化のグラフ描画

In [ ]:
plt.plot(rice["調査年"], rice["支出金額"], marker='o')
plt.title("各年の米の支出金額")
plt.xlabel("年")
plt.ylabel("支出金額（円）")
y_max = max(rice["支出金額"])
plt.ylim(0, y_max * 1.1)
plt.show()

[目次へ戻る](#contents)

<a class="anchor" id="code04-5"></a>
#### コード4.5 米の支出金額の変化率の計算とグラフの描画

In [ ]:
# 変化率の計算結果の表示
print(rice["支出金額"].pct_change())

# 変化率のグラフを描画する
plt.plot(rice["調査年"], rice["支出金額"].pct_change() * 100, marker='o')
plt.title("各年の米の支出金額の変化率")
plt.xlabel("年")
plt.ylabel("支出金額の変化率（％）")
plt.axhline(0, color="black")  # 変化率が 0 の位置に水平線を描画
plt.show()

[目次へ戻る](#contents)

---

<a class="anchor" id="section04-2b"></a>
**b. 他項目と比較を行うケース：米と他の項目との支出金額の傾向比較**

<a class="anchor" id="code04-6"></a>
#### コード4.6 食料の支出金額の集計処理

In [ ]:
import pandas as pd
kakei = pd.read_csv("kakei.csv", encoding="shift_jis")

# 調査年ごとに収支分類区分1（食料）で支出金額を合計
foods = kakei.groupby(["収支分類区分1", "調査年"], as_index=False).agg({"支出金額":sum})
foods

[目次へ戻る](#contents)

<a class="anchor" id="code04-7"></a>
#### コード4.7 穀類の支出金額の集計処理

In [ ]:
# 穀類の行だけ選択して、調査年ごとに収支分類区分2で支出金額を合計
cereals = kakei[kakei["収支分類区分2"] == "穀類"]
cereals = cereals.groupby(["収支分類区分1", "収支分類区分2", "調査年"], as_index=False).agg({"支出金額":sum})
cereals

[目次へ戻る](#contents)

---

<a class="anchor" id="section04-2b_comp"></a>
**米と他項目との比較**

<a class="anchor" id="code04-8"></a>
#### コード4.8 時系列データの基礎的な分析の関数化

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

def analyze(df, kubun_name):
    """
    支出金額データを時系列分析する関数
    df: 分析対象のデータ
    kubun_name: 分析対象の支出分類区分の名前
    """
    # 要約統計量
    print(df["支出金額"].describe())

    # データの分布
    plt.hist(df["支出金額"], bins=5, rwidth=0.9)
    plt.title(kubun_name + "の支出金額のヒストグラム")
    plt.xlabel("支出金額（円）")
    plt.ylabel("度数")
    plt.show()

    # 年ごとの支出金額
    plt.plot(df["調査年"], df["支出金額"], marker='o')
    plt.title("各年の"+ kubun_name + "の支出金額")
    plt.xlabel("年")
    plt.ylabel(kubun_name + "の支出金額（円）")
    y_max = max(df["支出金額"])
    plt.ylim(0, y_max * 1.1)
    plt.show()

[目次へ戻る](#contents)

<a class="anchor" id="code04-9"></a>
#### コード4.9 関数 analyze を使った基礎的な分析

In [ ]:
kakei = pd.read_csv("kakei.csv", encoding="Shift_JIS")

# 食料の基礎的な分析
foods = kakei.groupby(["収支分類区分1", "調査年"], as_index=False).agg({"支出金額":sum})
analyze(foods, "食料")

# 穀類の基礎的な分析
cereals = kakei[kakei["収支分類区分2"] == "穀類"]
cereals = cereals.groupby(["収支分類区分1", "収支分類区分2", "調査年"], as_index=False).agg({"支出金額":sum})
analyze(cereals, "穀類")

# 米の基礎的な分析
rice = kakei[kakei["収支分類区分3"] == "米"]
analyze(rice, "米")

[目次へ戻る](#contents)

<a class="anchor" id="code04-10"></a>
#### コード4.10 変化率を比較するグラフの描画

In [ ]:
# それぞれの変化率を折れ線グラフで表示する
plt.plot(rice["調査年"], rice["支出金額"].pct_change() * 100, marker='o', label="米")
plt.plot(cereals["調査年"], cereals["支出金額"].pct_change() * 100, marker='o', label="穀類")
plt.plot(foods["調査年"], foods["支出金額"].pct_change() * 100, marker='o', label="食料")
plt.title("支出金額の変化率の比較")
plt.xlabel("年")
plt.ylabel("支出金額の変化率（％）")
plt.legend(bbox_to_anchor=(1, -0.1), loc='upper right')
plt.axhline(0, color="black")
plt.show()

# それぞれの平均変化率を求める
mean_foods = foods["支出金額"].pct_change().mean() * 100
mean_cereals = cereals["支出金額"].pct_change().mean() * 100
mean_rice = rice["支出金額"].pct_change().mean() * 100
means = {"米":mean_rice, "穀類":mean_cereals, "食料":mean_foods}
## 棒グラフで表示する
plt.bar(x = means.keys(), height = means.values(), width = 0.7)
plt.title("支出金額の平均変化率の比較")
plt.xlabel("収支分類区分")
plt.ylabel("支出金額の平均変化率（％）")
plt.axhline(0, color="black")
plt.show()

[目次へ戻る](#contents)

---

<a class="anchor" id="section04-3"></a>
### 4.3 来年はどうなるの？

<a class="anchor" id="section04-3a"></a>
**a. 移動平均法を用いた傾向の可視化**

<a class="anchor" id="code04-11"></a>
#### コード4.11 支出金額の移動平均の計算

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

# 家計調査年報データを読み込む
kakei = pd.read_csv("kakei.csv", encoding="Shift_JIS")

# 米の支出金額のみを取り出す
rice = kakei[kakei["収支分類区分3"] == "米"]

# 項数に応じて期間の中心の年の支出金額の移動平均を計算
rice.index = rice["調査年"]
n = int(input("移動平均の項数を入力してください："))
ma_rice = rice["支出金額"].rolling(n, center=True).mean()
print(ma_rice)

# 年ごとの支出金額
## 元データの描画設定
plt.plot(rice.index, rice["支出金額"], label="生データ", marker='o')
## 移動平均を求めたデータの描画設定
plt.plot(ma_rice.index, ma_rice, label="移動平均", marker='o')
## グラフの要素設定
plt.title("各年の支出金額の移動平均")
plt.xlabel("年")
plt.ylabel("支出金額（円）")
plt.ylim(0, 2600)
plt.legend(bbox_to_anchor=(1, -0.1), loc='upper right')
plt.show()

[目次へ戻る](#contents)

<a class="anchor" id="code04-12"></a>
#### コード4.12 支出金額の変化率の移動平均の計算

In [ ]:
# 支出金額の変化率を計算する
raw = rice["支出金額"].pct_change() * 100
ma_change = ma_rice.pct_change() * 100
ma_change[2018] = None # 変化率の計算で最後の年が 0 になるため無効にする
print(ma_change)

## 変化率のグラフを描画する
plt.plot(raw.index, raw, label="生データ", marker='o')
plt.plot(ma_change.index, ma_change, label="移動平均", marker = 'o')
plt.title("各年の米の支出金額の変化率の移動平均")
plt.xlabel("年")
plt.ylabel("支出金額の変化率（％）")
plt.legend(bbox_to_anchor=(1, -0.1), loc='upper right')
plt.axhline(0, color="black")
plt.show()

[目次へ戻る](#contents)

---

<a class="anchor" id="section04-3b"></a>
**b. 指数平滑法を用いた予測モデルの構築**

<a class="anchor" id="code04-13"></a>
#### コード4.13 指数平滑法による予測値を計算する関数

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

def calc_forecasted_value(df, alpha):
    """
    指数平滑法を用いて予測値を計算する関数
    """
    # データフレームのコピーを作成
    res = df.copy()
    # 時間の列をインデックスに設定
    res.index = res["調査年"]
    
    # 予測値を格納する辞書（キー：調査年）
    forecasted = {res.index[0]: np.nan}  # 最初の年は予測できないので NaN
    forecasted[res.index[1]] = res.at[res.index[0], "支出金額"]  # 最初の年の値を次の年の予測値とする
    # 予測値を順次求めていく
    for t in range(2, len(res)):
        pre_year = res.index[t - 1]
        year_t = res.index[t]
        forecasted[year_t] = alpha * res.at[pre_year, "支出金額"] + (1 - alpha) * forecasted[pre_year]
    
    # 辞書の値（予測値）を列として追加
    res.loc[:, "予測値"] = forecasted.values()
    # 予測値と実測値から予測誤差を計算
    res["予測誤差"] = abs(res["支出金額"] - res["予測値"])
    
    return res

[目次へ戻る](#contents)

<a class="anchor" id="code04-14"></a>
#### コード4.14 係数 $ \alpha $ ごとの予測誤差の合計値の計算

In [ ]:
# 家計調査年報データを読み込む
kakei = pd.read_csv("kakei.csv", encoding="Shift_JIS")

# 米の支出金額のみを取り出す
rice = kakei[kakei["収支分類区分3"] == "米"]

# 予測のパラメータ α の候補
param_alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# パラメータごとの予測誤差の合計の辞書
total_error = {}
# パラメータを変えながら予測
for alpha in param_alpha:
    ## 予測値を計算する
    rice_f = calc_forecasted_value(rice, alpha)
    ## 予測誤差の合計値を計算して辞書へ追加
    te = sum(rice_f.loc[rice_f.index[1:], "予測誤差"])  # 最初の年は NaN なので除外
    total_error[str(alpha)] = te  # αの値を文字列化して辞書のキーとする
    
    """
    ## - - この部分のコメントアウトを外すとαごとの予測値のグラフを描画 - -
    ## 年ごとの消費支出と予測値を比較するグラフを描画する
    ### 元データの描画設定
    plt.plot(rice_f.index, rice_f["支出金額"], label="実測値", marker='o')
    ### 予測値の描画設定
    plt.plot(rice_f.index, rice_f["予測値"], label="予測値 (α = " + str(alpha) + ")", marker='o')
    ### グラフの要素設定
    plt.title("各年の米の支出金額")
    plt.xlabel("年")
    plt.ylabel("支出金額（円）")
    plt.ylim(0, 2600)
    plt.legend(bbox_to_anchor=(1, -0.1), loc='upper right')
    plt.show()
    """

# グラフ描画のために pandas.Series へ変換
res = pd.Series(total_error)
# パラメータ α ごとの予測誤差を描画
plt.plot(res.index, res, marker='o')
plt.title("パラメーターαごとの実測値と予測値との誤差の合計")
plt.xlabel("パラメーターα")
plt.ylabel("予測誤差の合計")
plt.ylim(0, max(res) * 1.1)
plt.show()

[目次へ戻る](#contents)

<a class="anchor" id="code04-15"></a>
#### コード4.15 次の年の支出金額の予測

In [ ]:
# αを入力して予測値を計算する
alpha = float(input("予測パラメーターα (0 < α < 1) を入力してください ==> "))

kakei = pd.read_csv("kakei.csv", encoding="Shift_JIS")
rice = kakei[kakei["収支分類区分3"] == "米"]
rice_f = calc_forecasted_value(rice, alpha)

# 年ごとの支出金額と予測値を比較するグラフを描画する
plt.plot(rice_f.index, rice_f["支出金額"], label="実測値", marker='o')
plt.plot(rice_f.index, rice_f["予測値"], label="予測値 (α = " + str(alpha) + ")", marker='o')
## グラフの要素設定
plt.title("各年の米の支出金額")
plt.xlabel("年")
plt.ylabel("支出金額（円）")
plt.ylim(0, max(rice_f["支出金額"]) * 1.1)
plt.legend(bbox_to_anchor=(1, -0.1), loc='upper right')
plt.show()

# 年ごとの実測値における絶対誤差の割合（誤差率）を描画する
### 元データの描画設定
plt.plot(rice_f.index, rice_f["予測誤差"] / rice_f["支出金額"] * 100, marker='o')
plt.title("各年の予測誤差率")
plt.xlabel("年")
plt.ylabel("予測誤差率（％）")
plt.show()

# 次の年の消費支出を予測する
## 最新データの行番号
last_year = rice_f.index[-1]
## 予測値を計算する
forecasted = alpha * rice_f.at[last_year, "支出金額"] + (1 - alpha) * rice_f.at[last_year, "予測値"]
## 計算結果を表示する
print("α = " + str(alpha) + "のとき：")
print("2019年の米の消費支出の予測値 = " + str(forecasted))

ans = 1857  # 2019年の実測値
print("2019年の実測値との誤差 = " + str(abs(ans - forecasted)))
print("2019年の実測値との誤差率（%） = " + str(abs(ans - forecasted) / ans * 100))

[目次へ戻る](#contents)